In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!wget https://dlcdn.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz

--2021-11-29 00:06:32--  https://dlcdn.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 220400553 (210M) [application/x-gzip]
Saving to: ‘spark-3.0.3-bin-hadoop2.7.tgz’

spark-3.0.3-bin-had 100%[===================>] 210.19M   144MB/s    in 1.5s    

2021-11-29 00:06:43 (144 MB/s) - ‘spark-3.0.3-bin-hadoop2.7.tgz’ saved [220400553/220400553]



In [3]:
!tar xf /content/spark-3.0.3-bin-hadoop2.7.tgz

In [4]:
!pip install -q findspark

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"

In [36]:
import findspark
from pyspark.sql import Row
findspark.init("/content/spark-3.0.3-bin-hadoop2.7")# SPARK_HOME
from pyspark.sql import SparkSession
ss = SparkSession.builder.master("local[*]").getOrCreate()
SpContext = ss.sparkContext

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [25]:
PATH = '/content/drive/MyDrive/BigData/dataset\ proyecto/myfile.csv'
df = ss.read.csv(PATH, header=True)

In [26]:
df = df.select('id', 'labels', 'tweet_text', 'labels_str')
df.take(3)

[Row(id='1114679353714016256', labels='[4, 1, 3]', tweet_text='@FriskDontMiss Nigga https://t.co/cAsaLWEpue', labels_str="['Religion', 'Racist', 'Homophobe']"),
 Row(id='1063020048816660480', labels='[5, 5, 5]', tweet_text='My horses are retarded https://t.co/HYhqc6d5WN', labels_str="['OtherHate', 'OtherHate', 'OtherHate']"),
 Row(id='1108927368075374593', labels='[0, 0, 0]', tweet_text='“NIGGA ON MA MOMMA YOUNGBOY BE SPITTING REAL SHIT NIGGA” https://t.co/UczofqHrLq', labels_str="['NotHate', 'NotHate', 'NotHate']")]

In [38]:
_stopwords = set(stopwords.words("english") + ['\'', '’'])

def limpiar(row):
  t = row['tweet_text']
  without_links = ' '.join([word for word in t.split(' ') if word[:4] != 'http']) # quitar los links
  tokens = word_tokenize(without_links) # tokenizar (dividir en tokens)
  tokens = map(lambda token: token.encode('ascii', 'ignore').decode('ascii'), tokens)
  tokens_without_sw = [word.lower() for word in tokens if word.isalpha() and not word in _stopwords]
  new_row = Row(id=row['id'], labels=row['labels'], tweet_text=' '.join(tokens_without_sw), labels_str=row['labels_str'])
  return new_row

In [39]:
df_limpio = df.rdd.map(limpiar)
df_limpio.take(2)

[Row(id='1114679353714016256', labels='[4, 1, 3]', tweet_text='friskdontmiss nigga', labels_str="['Religion', 'Racist', 'Homophobe']"),
 Row(id='1063020048816660480', labels='[5, 5, 5]', tweet_text='my horses retarded', labels_str="['OtherHate', 'OtherHate', 'OtherHate']")]